In [3]:
import pandas as pd
import numpy as np
import re
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
listings_data = pd.read_csv('../data/listings.csv.gz', compression='gzip')

In [5]:
listings_data

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3781,https://www.airbnb.com/rooms/3781,20250315060211,2025-03-15,city scrape,HARBORSIDE-Walk to subway,Fully separate apartment in a two apartment bu...,"Mostly quiet ( no loud music, no crowed sidewa...",https://a0.muscache.com/pictures/24670/b2de044...,4804,...,4.96,4.85,4.88,NaN,f,1,1,0,0,0.22
1,5506,https://www.airbnb.com/rooms/5506,20250315060211,2025-03-15,city scrape,** Fort Hill Inn Private! Minutes to center!**,**THE BEST Value in BOSTON!!*** PRIVATE GUEST ...,"Peaceful, Architecturally interesting, histori...",https://a0.muscache.com/pictures/miso/Hosting-...,8229,...,4.89,4.56,4.76,STR-490093,f,11,11,0,0,0.68
2,6695,https://www.airbnb.com/rooms/6695,20250315060211,2025-03-15,city scrape,"Fort Hill Inn *Sunny* 1 bedroom, condo duplex","Comfortable, Fully Equipped private apartment...","Peaceful, Architecturally interesting, histori...",https://a0.muscache.com/pictures/38ac4797-e7a4...,8229,...,4.95,4.53,4.72,STR-491702,f,11,11,0,0,0.71
3,8789,https://www.airbnb.com/rooms/8789,20250315060211,2025-03-15,previous scrape,Curved Glass Studio/1bd facing Park,This unit is for sale. There will need to be o...,Beacon Hill is a historic neighborhood filled ...,https://a0.muscache.com/pictures/miso/Hosting-...,26988,...,4.97,4.97,4.59,NaN,f,6,6,0,0,0.22
4,10811,https://www.airbnb.com/rooms/10811,20250315060211,2025-03-15,city scrape,"Back Bay Apt Studio-3 blocks to Pru center & ""T""",Stunning Back Bay furnished studio apartment. ...,A one-square mile neighborhood that is arguabl...,https://a0.muscache.com/pictures/45735/27548f7...,38997,...,4.00,5.00,4.67,NaN,f,12,12,0,0,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4603,1375656401110940289,https://www.airbnb.com/rooms/1375656401110940289,20250315060211,2025-03-15,city scrape,Fabulous North End 3 BR | 1.5 BA in Little Italy,Newly renovated 3 Bedroom | 1.5 Bath ground-le...,The North End is Boston's oldest residential n...,https://a0.muscache.com/pictures/e41ad5a1-1966...,9410008,...,NaN,NaN,NaN,STR-544876,t,16,16,0,0,NaN
4604,1375662640245506811,https://www.airbnb.com/rooms/1375662640245506811,20250315060211,2025-03-15,city scrape,"Classy, Central Beacon Hill Penthouse - NEW",Enjoy a stylish experience at this centrally-l...,Located in an upscale historic district in dow...,https://a0.muscache.com/pictures/hosting/Hosti...,263259492,...,NaN,NaN,NaN,STR-586938,f,2,2,0,0,NaN
4605,1375681174270903038,https://www.airbnb.com/rooms/1375681174270903038,20250315060211,2025-03-15,city scrape,Luxury 3brm/1bath Duplex in JP,This thoughtfully furnished and well designed ...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,2006852,...,NaN,NaN,NaN,STR-584174,t,21,21,0,0,NaN
4606,1375695500454617370,https://www.airbnb.com/rooms/1375695500454617370,20250315060211,2025-03-15,city scrape,Large 3.5 BR|2 BA Penthouse Duplex,Just updated 3.5 BR | 2 BA condo that features...,Centrally located in Boston's historic Beacon ...,https://a0.muscache.com/pictures/83937472/0272...,9410008,...,NaN,NaN,NaN,STR-544866,t,16,16,0,0,NaN


In [6]:
listings_data.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [ ]:
listings_columns = [ "id", "name", "description", "host_id", "listing_url", "neighbourhood_cleansed","neighborhood_overview",
    "picture_url", "latitude", "longitude", "property_type", "room_type", "accommodates",
    "bathrooms","bathrooms_text", "bedrooms", "beds", "amenities", "license", 'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin',
    'review_scores_communication', 'review_scores_location',
    'review_scores_value','number_of_reviews']


In [ ]:
listings_df = listings_data[listings_columns].copy()
listings_df.rename(columns={'id': 'listing_id', 'review_scores_rating': 'overall_rating','review_scores_accuracy': 'accuracy_rating',
    'review_scores_cleanliness': 'cleanliness_rating', 'review_scores_checkin': 'checkin_rating',
    'review_scores_communication': 'communication_rating', 'review_scores_location': 'location_rating',
    'review_scores_value': 'value_rating'}, inplace=True)

In [9]:
listings_df

,listing_id,name,description,host_id,listing_url,neighbourhood_cleansed,neighborhood_overview,picture_url,latitude,longitude,...,amenities,license,overall_rating,accuracy_rating,cleanliness_rating,checkin_rating,communication_rating,location_rating,value_rating,number_of_reviews
0,3781,HARBORSIDE-Walk to subway,Fully separate apartment in a two apartment bu...,4804,https://www.airbnb.com/rooms/3781,East Boston,"Mostly quiet ( no loud music, no crowed sidewa...",https://a0.muscache.com/pictures/24670/b2de044...,42.364130,-71.029910,...,"[""Oven"", ""Extra pillows and blankets"", ""Cookin...",NaN,4.96,5.00,4.96,5.00,4.96,4.85,4.88,26
1,5506,** Fort Hill Inn Private! Minutes to center!**,**THE BEST Value in BOSTON!!*** PRIVATE GUEST ...,8229,https://www.airbnb.com/rooms/5506,Roxbury,"Peaceful, Architecturally interesting, histori...",https://a0.muscache.com/pictures/miso/Hosting-...,42.328440,-71.095810,...,"[""Extra pillows and blankets"", ""Hair dryer"", ""...",STR-490093,4.81,4.88,4.91,4.95,4.89,4.56,4.76,132
2,6695,"Fort Hill Inn *Sunny* 1 bedroom, condo duplex","Comfortable, Fully Equipped private apartment...",8229,https://www.airbnb.com/rooms/6695,Roxbury,"Peaceful, Architecturally interesting, histori...",https://a0.muscache.com/pictures/38ac4797-e7a4...,42.328020,-71.093870,...,"[""Oven"", ""Cooking basics"", ""Hair dryer"", ""Kitc...",STR-491702,4.81,4.83,4.86,4.91,4.95,4.53,4.72,135
3,8789,Curved Glass Studio/1bd facing Park,This unit is for sale. There will need to be o...,26988,https://www.airbnb.com/rooms/8789,Beacon Hill,Beacon Hill is a historic neighborhood filled ...,https://a0.muscache.com/pictures/miso/Hosting-...,42.358670,-71.063070,...,"[""Extra pillows and blankets"", ""Cooking basics...",NaN,4.69,4.69,4.55,4.93,4.97,4.97,4.59,29
4,10811,"Back Bay Apt Studio-3 blocks to Pru center & ""T""",Stunning Back Bay furnished studio apartment. ...,38997,https://www.airbnb.com/rooms/10811,Back Bay,A one-square mile neighborhood that is arguabl...,https://a0.muscache.com/pictures/45735/27548f7...,42.351730,-71.086850,...,"[""Hair dryer"", ""Kitchen"", ""Essentials"", ""Pets ...",NaN,4.33,4.44,4.67,4.56,4.00,5.00,4.67,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4603,1375656401110940289,Fabulous North End 3 BR | 1.5 BA in Little Italy,Newly renovated 3 Bedroom | 1.5 Bath ground-le...,9410008,https://www.airbnb.com/rooms/1375656401110940289,North End,The North End is Boston's oldest residential n...,https://a0.muscache.com/pictures/e41ad5a1-1966...,42.367523,-71.052688,...,"[""Oven"", ""Extra pillows and blankets"", ""Cookin...",STR-544876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4604,1375662640245506811,"Classy, Central Beacon Hill Penthouse - NEW",Enjoy a stylish experience at this centrally-l...,263259492,https://www.airbnb.com/rooms/1375662640245506811,Beacon Hill,Located in an upscale historic district in dow...,https://a0.muscache.com/pictures/hosting/Hosti...,42.357720,-71.071090,...,"[""Oven"", ""Extra pillows and blankets"", ""Hair d...",STR-586938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4605,1375681174270903038,Luxury 3brm/1bath Duplex in JP,This thoughtfully furnished and well designed ...,2006852,https://www.airbnb.com/rooms/1375681174270903038,Jamaica Plain,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,42.318576,-71.106661,...,"[""Oven"", ""Extra pillows and blankets"", ""Cookin...",STR-584174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4606,1375695500454617370,Large 3.5 BR|2 BA Penthouse Duplex,Just updated 3.5 BR | 2 BA condo that features...,9410008,https://www.airbnb.com/rooms/1375695500454617370,Beacon Hill,Centrally located in Boston's historic Beacon ...,https://a0.muscache.com/pictures/83937472/0272...,42.360268,-71.069963,...,"[""Oven"", ""Extra pillows and blankets"", ""Cookin...",STR-544866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
# Filling in missing values
listings_df['description'] = listings_df['description'].fillna('No description given')
listings_df['neighborhood_overview'] = listings_df['neighborhood_overview'].fillna('No neighborhood overview given')
listings_df['license'] = listings_df['license'].fillna('No license information')

listings_df['bathrooms'] = np.where(listings_df['bathrooms'].isnull(), listings_df['bathrooms_text'].str.extract(r'(\d+\.?\d*)').astype(float), listings_df['bathrooms'])

listings_df['bathrooms'] = listings_df['bathrooms'].fillna(0) 
listings_df['bedrooms'] = listings_df['bedrooms'].fillna(1)  
listings_df['beds'] = listings_df['beds'].fillna(1)

listings_df['bathroom_type'] = np.where(listings_df['bathrooms_text'].str.contains('shared', case=False, na=False), 'shared', 'private')


def clean_and_split_amenities(text):
    if not isinstance(text, str) or not text.strip():
        return ['no amenities listed']
    
    text = text.lower()
    
    text = re.sub(r'[\[\]"]', '', text)  # Remove brackets and quotes  
                   
    text = re.sub(r'[^\w\s,]', '', text) # Remove punctuation like &?! etc.           
             
    text = re.sub(r'\band\b', ',', text) # Replace 'and' with comma 
    
    text = re.sub(r'\s*,\s*', ', ', text) # Normalize comma spacing      
             
    text = re.sub(r'\s+', ' ', text).strip() # Normalize whitespace               

    if text == '':
        return ['no amenities listed']
    
    return [a.strip() for a in text.split(',') if a.strip()]

listings_df['amenities'] = listings_df['amenities'].apply(clean_and_split_amenities)


listings_df.drop(columns=['bathrooms_text'], inplace=True)

In [11]:
listings_df

,listing_id,name,description,host_id,listing_url,neighbourhood_cleansed,neighborhood_overview,picture_url,latitude,longitude,...,license,overall_rating,accuracy_rating,cleanliness_rating,checkin_rating,communication_rating,location_rating,value_rating,number_of_reviews,bathroom_type
0,3781,HARBORSIDE-Walk to subway,Fully separate apartment in a two apartment bu...,4804,https://www.airbnb.com/rooms/3781,East Boston,"Mostly quiet ( no loud music, no crowed sidewa...",https://a0.muscache.com/pictures/24670/b2de044...,42.364130,-71.029910,...,No license information,4.96,5.00,4.96,5.00,4.96,4.85,4.88,26,private
1,5506,** Fort Hill Inn Private! Minutes to center!**,**THE BEST Value in BOSTON!!*** PRIVATE GUEST ...,8229,https://www.airbnb.com/rooms/5506,Roxbury,"Peaceful, Architecturally interesting, histori...",https://a0.muscache.com/pictures/miso/Hosting-...,42.328440,-71.095810,...,STR-490093,4.81,4.88,4.91,4.95,4.89,4.56,4.76,132,private
2,6695,"Fort Hill Inn *Sunny* 1 bedroom, condo duplex","Comfortable, Fully Equipped private apartment...",8229,https://www.airbnb.com/rooms/6695,Roxbury,"Peaceful, Architecturally interesting, histori...",https://a0.muscache.com/pictures/38ac4797-e7a4...,42.328020,-71.093870,...,STR-491702,4.81,4.83,4.86,4.91,4.95,4.53,4.72,135,private
3,8789,Curved Glass Studio/1bd facing Park,This unit is for sale. There will need to be o...,26988,https://www.airbnb.com/rooms/8789,Beacon Hill,Beacon Hill is a historic neighborhood filled ...,https://a0.muscache.com/pictures/miso/Hosting-...,42.358670,-71.063070,...,No license information,4.69,4.69,4.55,4.93,4.97,4.97,4.59,29,private
4,10811,"Back Bay Apt Studio-3 blocks to Pru center & ""T""",Stunning Back Bay furnished studio apartment. ...,38997,https://www.airbnb.com/rooms/10811,Back Bay,A one-square mile neighborhood that is arguabl...,https://a0.muscache.com/pictures/45735/27548f7...,42.351730,-71.086850,...,No license information,4.33,4.44,4.67,4.56,4.00,5.00,4.67,9,private
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4603,1375656401110940289,Fabulous North End 3 BR | 1.5 BA in Little Italy,Newly renovated 3 Bedroom | 1.5 Bath ground-le...,9410008,https://www.airbnb.com/rooms/1375656401110940289,North End,The North End is Boston's oldest residential n...,https://a0.muscache.com/pictures/e41ad5a1-1966...,42.367523,-71.052688,...,STR-544876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,private
4604,1375662640245506811,"Classy, Central Beacon Hill Penthouse - NEW",Enjoy a stylish experience at this centrally-l...,263259492,https://www.airbnb.com/rooms/1375662640245506811,Beacon Hill,Located in an upscale historic district in dow...,https://a0.muscache.com/pictures/hosting/Hosti...,42.357720,-71.071090,...,STR-586938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,private
4605,1375681174270903038,Luxury 3brm/1bath Duplex in JP,This thoughtfully furnished and well designed ...,2006852,https://www.airbnb.com/rooms/1375681174270903038,Jamaica Plain,No neighborhood overview given,https://a0.muscache.com/pictures/miso/Hosting-...,42.318576,-71.106661,...,STR-584174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,private
4606,1375695500454617370,Large 3.5 BR|2 BA Penthouse Duplex,Just updated 3.5 BR | 2 BA condo that features...,9410008,https://www.airbnb.com/rooms/1375695500454617370,Beacon Hill,Centrally located in Boston's historic Beacon ...,https://a0.muscache.com/pictures/83937472/0272...,42.360268,-71.069963,...,STR-544866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,private


In [12]:
# Extract unique amenities
all_amenities = sorted(set(amenity for sublist in listings_df['amenities'] for amenity in sublist if amenity.strip()))

In [13]:
all_amenities

['10 inch hdtv',
 '10 years old',
 '100 vegan crueltyfree',
 '19 inch hdtv with amazon prime video',
 '2 burner cook top only electric stove',
 '24 inch hdtv',
 '24 inch tv',
 '25 years old',
 '28 inch hdtv with amazon prime video',
 '30 inch hdtv',
 '30 inch hdtv with amazon prime video',
 '32 inch hdtv with amazon prime video',
 '32 inch hdtv with apple tv',
 '32 inch hdtv with fire tv',
 '32 inch hdtv with netflix',
 '32 inch hdtv with roku',
 '32 inch hdtv with standard cable',
 '32 inch tv',
 '32 inch tv with fire tv',
 '32 inch tv with netflix',
 '32 inch tv with roku',
 '34 inch hdtv with roku',
 '35 inch hdtv with amazon prime video',
 '36 inch hdtv',
 '36 inch hdtv with apple tv',
 '36 inch hdtv with chromecast',
 '36 inch hdtv with fire tv',
 '365 body soap',
 '37 inch hdtv with amazon prime video',
 '38 inch hdtv with amazon prime video',
 '4 inch hdtv with roku',
 '40 inch hdtv',
 '40 inch hdtv with amazon prime video',
 '40 inch hdtv with apple tv',
 '40 inch hdtv with chr

In [ ]:

amenity_category_map = {
    'TV': ['hdtv', 'tv', 'screen', 'television', 'smart tv', 'flat screen', 'oled', 'led'],
    'Streaming': ['netflix', 'amazon prime', 'fire tv', 'apple tv', 'roku', 'hulu', 'disney', 'hbo', 'streaming service', 'streaming'],
    'Kitchen': ['oven', 'stove', 'refrigerator', 'kitchen', 'cooktop'],
    'Air Conditioning': ['air conditioning', 'ac', 'central air', 'split type ductless'],
    'Heating': ['heating', 'heater'],
    'View': ['beach view', 'lake view', 'mountain view', 'city skyline view', 'garden view', 'view'],
    'Parking': ['parking', 'garage', 'driveway', 'carport'],
    'Internet': ['wifi', 'ethernet', 'internet'],
    'Toiletries': ['soap', 'shampoo', 'conditioner', 'body wash', 'toiletries'],
    'Kitchen Appliances': ['dishwasher', 'kettle', 'coffee maker', 'microwave', 'toaster'],
    'Other Appliances': ['iron','hair dryer'],
    'Essentials': ['towels', 'linens', 'toilet paper', 'paper towels', 'kitchen essentials'],
    'Safety Equipemnt': ['smoke alarm', 'carbon monoxide alarm', 'fire extinguisher', 'first aid kit'],
    'Bedding': ['sheets', 'pillows', 'blankets', 'comforter', 'bedding'],
    'Dining': ['dining table', 'dining area', 'bar stools', 'kitchen island', 'cutlery', 'dishes', 'glassware', 'silverware', 'pots and pans'],
    'Laundry': ['washer', 'dryer', 'laundromat', 'laundry'],
    'Gym': ['gym', 'exercise equipment', 'weights', 'treadmill'],
    'Entertainment': ['board games', 'arcade', 'pool table', 'ping pong', 'dvd player', 'game console'],
    'Pool': ['pool', 'lap pool', 'indoor pool', 'outdoor pool','hot tub'],
    'Fireplace': ['fireplace'],
    'Baby Friendly': ['crib', 'high chair', 'baby monitor', 'booster seat', 'changing table', 'baby friendly'],
    'Pet Friendly': ['pets allowed', 'pet friendly', 'cat(s)', 'dog(s)', 'pet(s)'],
    'Smoking': ['smoking allowed', 'smoking'],
    'Security': ['security system', 'doorman', 'safe', 'lockbox', 'keypad'],
    'Outdoor Space': ['balcony', 'terrace', 'patio', 'garden', 'deck', 'outdoor furniture', 'outdoor space'],
    'Accessibility': ['wheelchair accessible', 'elevator', 'step-free access', 'grab bars'],
    'Miscellaneous': ['miscellaneous', 'other', 'various amenities'],
    'No Amenities': ['no amenities listed']
}


category_texts = [' '.join(keywords) for keywords in amenity_category_map.values()]
category_labels = list(amenity_category_map.keys())

print("Extracting unique amenities...")
unique_amenities = set()
for amenities_list in listings_df['amenities']:
    unique_amenities.update(amenities_list)
unique_amenities = list(unique_amenities)


print("Fitting TF-IDF vectorizer...")
vectorizer = TfidfVectorizer().fit(category_texts + unique_amenities)

# Encode categories and amenities
category_vectors = vectorizer.transform(category_texts)
amenity_vectors = vectorizer.transform(unique_amenities)

# Map amenity to vector
amenity_to_vector = dict(zip(unique_amenities, amenity_vectors))


def match_amenity_to_category_tfidf(amenity_str, category_vectors, category_labels):
    v = amenity_to_vector.get(amenity_str)
    
    if v is None or v.nnz == 0:
        return ['Miscellaneous']
    sims = cosine_similarity(v, category_vectors)[0]
    matches = [category_labels[i] for i, score in enumerate(sims) if score > 0.2] 
    return matches if matches else ['Miscellaneous']


print("Matching amenities to categories...")
listings_df['amenity_categories'] = listings_df['amenities'].apply( lambda amenities: list(set(cat for amenity in amenities for cat in match_amenity_to_category_tfidf(amenity, category_vectors, category_labels))))


Extracting unique amenities...
Fitting TF-IDF vectorizer...
Matching amenities to categories...


In [15]:
unique_amenity_categories = list(amenity_category_map.keys())

amenity_df = pd.DataFrame({
    'amenity_id': range(1, len(unique_amenity_categories) + 1),
    'amenity_name': unique_amenity_categories
})

In [16]:
amenity_df

,amenity_id,amenity_name
0,1,TV
1,2,Streaming
2,3,Kitchen
3,4,Air Conditioning
4,5,Heating
5,6,View
6,7,Parking
7,8,Internet
8,9,Toiletries
9,10,Kitchen Appliances


In [17]:
# amenity_df.to_csv('../data/cleaned_amenities.csv', index=False)

In [18]:
amenity_to_id = dict(zip(amenity_df['amenity_name'], amenity_df['amenity_id']))


In [ ]:
listing_amenities = [(listing_id, amenity_to_id[amenity]) for listing_id, amenities in zip(listings_df['listing_id'], listings_df['amenity_categories']) for amenity in amenities]

listing_amenities_df = pd.DataFrame(listing_amenities, columns=['listing_id', 'amenity_id'])


In [20]:
listing_amenities_df

,listing_id,amenity_id
0,3781,16
1,3781,10
2,3781,12
3,3781,9
4,3781,25
...,...,...
70254,1376287116576861534,13
70255,1376287116576861534,3
70256,1376287116576861534,1
70257,1376287116576861534,4


In [21]:
# listing_amenities_df.to_csv('../data/cleaned_listing_amenities.csv', index=False)

In [22]:
listings_df

,listing_id,name,description,host_id,listing_url,neighbourhood_cleansed,neighborhood_overview,picture_url,latitude,longitude,...,overall_rating,accuracy_rating,cleanliness_rating,checkin_rating,communication_rating,location_rating,value_rating,number_of_reviews,bathroom_type,amenity_categories
0,3781,HARBORSIDE-Walk to subway,Fully separate apartment in a two apartment bu...,4804,https://www.airbnb.com/rooms/3781,East Boston,"Mostly quiet ( no loud music, no crowed sidewa...",https://a0.muscache.com/pictures/24670/b2de044...,42.364130,-71.029910,...,4.96,5.00,4.96,5.00,4.96,4.85,4.88,26,private,"[Laundry, Kitchen Appliances, Essentials, Toil..."
1,5506,** Fort Hill Inn Private! Minutes to center!**,**THE BEST Value in BOSTON!!*** PRIVATE GUEST ...,8229,https://www.airbnb.com/rooms/5506,Roxbury,"Peaceful, Architecturally interesting, histori...",https://a0.muscache.com/pictures/miso/Hosting-...,42.328440,-71.095810,...,4.81,4.88,4.91,4.95,4.89,4.56,4.76,132,private,"[Laundry, Kitchen Appliances, Essentials, Toil..."
2,6695,"Fort Hill Inn *Sunny* 1 bedroom, condo duplex","Comfortable, Fully Equipped private apartment...",8229,https://www.airbnb.com/rooms/6695,Roxbury,"Peaceful, Architecturally interesting, histori...",https://a0.muscache.com/pictures/38ac4797-e7a4...,42.328020,-71.093870,...,4.81,4.83,4.86,4.91,4.95,4.53,4.72,135,private,"[Laundry, Kitchen Appliances, Essentials, Toil..."
3,8789,Curved Glass Studio/1bd facing Park,This unit is for sale. There will need to be o...,26988,https://www.airbnb.com/rooms/8789,Beacon Hill,Beacon Hill is a historic neighborhood filled ...,https://a0.muscache.com/pictures/miso/Hosting-...,42.358670,-71.063070,...,4.69,4.69,4.55,4.93,4.97,4.97,4.59,29,private,"[Accessibility, Laundry, Kitchen Appliances, E..."
4,10811,"Back Bay Apt Studio-3 blocks to Pru center & ""T""",Stunning Back Bay furnished studio apartment. ...,38997,https://www.airbnb.com/rooms/10811,Back Bay,A one-square mile neighborhood that is arguabl...,https://a0.muscache.com/pictures/45735/27548f7...,42.351730,-71.086850,...,4.33,4.44,4.67,4.56,4.00,5.00,4.67,9,private,"[Laundry, Kitchen Appliances, Essentials, Toil..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4603,1375656401110940289,Fabulous North End 3 BR | 1.5 BA in Little Italy,Newly renovated 3 Bedroom | 1.5 Bath ground-le...,9410008,https://www.airbnb.com/rooms/1375656401110940289,North End,The North End is Boston's oldest residential n...,https://a0.muscache.com/pictures/e41ad5a1-1966...,42.367523,-71.052688,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,private,"[Laundry, Kitchen Appliances, Essentials, Toil..."
4604,1375662640245506811,"Classy, Central Beacon Hill Penthouse - NEW",Enjoy a stylish experience at this centrally-l...,263259492,https://www.airbnb.com/rooms/1375662640245506811,Beacon Hill,Located in an upscale historic district in dow...,https://a0.muscache.com/pictures/hosting/Hosti...,42.357720,-71.071090,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,private,"[Miscellaneous, Baby Friendly, Laundry, Kitche..."
4605,1375681174270903038,Luxury 3brm/1bath Duplex in JP,This thoughtfully furnished and well designed ...,2006852,https://www.airbnb.com/rooms/1375681174270903038,Jamaica Plain,No neighborhood overview given,https://a0.muscache.com/pictures/miso/Hosting-...,42.318576,-71.106661,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,private,"[Laundry, Kitchen Appliances, Essentials, Toil..."
4606,1375695500454617370,Large 3.5 BR|2 BA Penthouse Duplex,Just updated 3.5 BR | 2 BA condo that features...,9410008,https://www.airbnb.com/rooms/1375695500454617370,Beacon Hill,Centrally located in Boston's historic Beacon ...,https://a0.muscache.com/pictures/83937472/0272...,42.360268,-71.069963,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,private,"[Laundry, Kitchen Appliances, Essentials, Toil..."


In [23]:
hosts_columns = ['host_id', 'host_name', 'host_url','host_since', 'host_location','host_about', 'host_response_time',
                 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost','host_thumbnail_url','host_picture_url',
                 'host_neighbourhood', 'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified','host_verifications']

hosts_df = listings_data[hosts_columns].copy()


In [24]:
hosts_df['host_neighbourhood'] = np.where((hosts_df['host_location'].isna())&(hosts_df['host_neighbourhood'].isna()), 'Unknown', hosts_df['host_neighbourhood'])
hosts_df['host_neighbourhood'] = hosts_df['host_neighbourhood'].fillna('Not Specified')

hosts_df['host_location'] = np.where((hosts_df['host_location'].isna()) & (hosts_df['host_neighbourhood'] == 'Unknown'), 'Unknown', hosts_df['host_location'])
hosts_df['host_location'] = hosts_df['host_location'].fillna('Boston, MA')

In [25]:
locations_df = hosts_df[['host_neighbourhood','host_location']].drop_duplicates().reset_index(drop=True).rename(columns={'host_neighbourhood': 'neighbourhood', 'host_location': 'location'})

In [26]:
listings_df['neighbourhood_cleansed'] = listings_df['neighbourhood_cleansed'].fillna('Not Specified')
listings_df['location'] = 'Boston, MA'

listings_location_df = listings_df[['neighbourhood_cleansed', 'location']].drop_duplicates().reset_index(drop=True).rename(columns={'neighbourhood_cleansed': 'neighbourhood', 'location': 'location'})

locations_df = pd.concat([locations_df, listings_location_df], ignore_index=True).drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index': 'location_id'})
locations_df['location_id'] = locations_df['location_id'] + 1

In [27]:
locations_df

,location_id,neighbourhood,location
0,1,East Boston,"Massachusetts, United States"
1,2,Roxbury,"Boston, MA"
2,3,Beacon Hill,"Boston, MA"
3,4,Back Bay,"Boston, MA"
4,5,Dorchester,"Boston, MA"
...,...,...,...
318,319,Sandwich,"Wayland, MA"
319,320,Palermo Hollywood,"Buenos Aires, Argentina"
320,321,Fenway,"Boston, MA"
321,322,South Boston Waterfront,"Boston, MA"


In [28]:
# locations_df.to_csv('../data/cleaned_locations.csv', index=False)

In [29]:
locations_neighborhood_map = dict(zip(locations_df['neighbourhood']+', '+ locations_df['location'], locations_df['location_id']))

In [30]:
locations_neighborhood_map

{'East Boston, Massachusetts, United States': 1,
 'Roxbury, Boston, MA': 2,
 'Beacon Hill, Boston, MA': 3,
 'Back Bay, Boston, MA': 4,
 'Dorchester, Boston, MA': 5,
 'Charlestown, Boston, MA': 6,
 'Jamaica Plain, Boston, MA': 7,
 'North End, Boston, MA': 8,
 'South Boston, Boston, MA': 9,
 'Allston-Brighton, Brookline, MA': 10,
 'Roslindale, Boston, MA': 11,
 'Downtown Crossing, Boston, MA': 12,
 'South End, Boston, MA': 13,
 'Government Center, Boston, MA': 14,
 'South End, Brookline, MA': 15,
 'West End, Boston, MA': 16,
 'Allston-Brighton, Boston, MA': 17,
 'Not Specified, Boston, MA': 18,
 'Roslindale, Massachusetts, United States': 19,
 'Fenway/Kenmore, Boston, MA': 20,
 'Allston-Brighton, New York, NY': 21,
 'Back Bay, Cambridge, MA': 22,
 'Hyde Park, Boston, MA': 23,
 'Flatbush, Boston, MA': 24,
 'Unknown, Unknown': 25,
 'West Roxbury, Boston, MA': 26,
 'East Boston, Boston, MA': 27,
 'Fenway/Kenmore, New York, NY': 28,
 'Beacon Hill, Massachusetts, United States': 29,
 'Not Spe

In [31]:
hosts_df['location_id'] = hosts_df.apply(lambda row: locations_neighborhood_map[row['host_neighbourhood'] + ', ' + row['host_location']], axis=1)

In [32]:
hosts_df.drop(columns=['host_neighbourhood', 'host_location'], inplace=True)

In [33]:
listings_df['location_id'] = listings_df.apply(lambda row: locations_neighborhood_map[row['neighbourhood_cleansed'] + ', ' + row['location']], axis=1)

In [35]:
listings_df.drop(columns=['neighbourhood_cleansed', 'location'], inplace=True)

In [39]:
listings_df = listings_df[['listing_id', 'name', 'description', 'host_id', 'listing_url',
       'location_id', 'neighborhood_overview', 'picture_url',
       'latitude', 'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'bathroom_type', 'beds', 'amenities', 'amenity_categories', 'license',
       'overall_rating', 'accuracy_rating', 'cleanliness_rating',
       'checkin_rating', 'communication_rating', 'location_rating',
       'value_rating', 'number_of_reviews']]

In [ ]:
# listings_df.to_csv('../data/cleaned_listings.csv', index=False)

In [42]:
hosts_df = hosts_df[['host_id', 'host_name', 'host_url', 'host_since', 'location_id', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_total_listings_count', 'host_verifications', 'host_has_profile_pic',
       'host_identity_verified']]

In [44]:
hosts_df['host_name'] = hosts_df['host_name'].fillna('Unknown Host')

In [45]:
hosts_df['host_is_superhost'] = hosts_df['host_is_superhost'].apply(lambda x: True if x == 't' else False)
hosts_df['host_has_profile_pic'] = hosts_df['host_has_profile_pic'].apply(lambda x: True if x == 't' else False)
hosts_df['host_identity_verified'] = hosts_df['host_identity_verified'].apply(lambda x: True if x == 't' else False)

In [ ]:
# hosts_df.to_csv('../data/cleaned_hosts.csv', index=False)

In [38]:
# calendar_data = pd.read_csv('../data/calendar.csv.gz', compression='gzip')